In [63]:
import pandas as pd
import numpy as np
import re
import string

In [64]:
# Load dataset with proper encoding
df = pd.read_excel(r'E:\OM Research Project\Combined_Data.xlsx', engine='openpyxl')
df.dropna(inplace=True)

In [65]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)  # Remove punctuation
    return text

In [66]:
df.head(4)

,Date,From,To,Subject,Body
0,"Tue, 19 Nov 2024 08:13:09 +0000 (UTC)",Career Navigator 2024* <newsletters-noreply@li...,Ajay Mane <ajaymane5885@gmail.com>,Infosys Walk-In for IT Operations | 21st Nov 2024,Career Navigator 2024*By AMISHA BHATTBy AMISHA...
1,"Wed, 20 Nov 2024 06:13:07 +0000 (UTC)",Work From Home Jobs <newsletters-noreply@linke...,Ajay Mane <ajaymane5885@gmail.com>,Genpact Jobs Gurgaon for Freshers| Best Opport...,Work From Home JobsBy Work From Home/ Experien...
2,"Tue, 19 Nov 2024 06:13:07 +0000 (UTC)",Work From Home Jobs <newsletters-noreply@linke...,Ajay Mane <ajaymane5885@gmail.com>,Genpact Jobs Noida for Freshers| Best Opportun...,Work From Home JobsBy Work From Home/ Experien...
3,"Mon, 18 Nov 2024 14:33:15 +0000 (UTC)",LinkedIn <updates-noreply@linkedin.com>,Ajay Mane <ajaymane5885@gmail.com>,Vaibhav Kadam just posted something that might...,---------------------------------------- ...


In [67]:
df['processed_text'] = df['Body'].apply(preprocess_text)

In [68]:
df.shape

(32016, 6)

In [69]:
df.isnull().sum()

Date              0
From              0
To                0
Subject           0
Body              0
processed_text    0
dtype: int64

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32016 entries, 0 to 55435
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            32016 non-null  object
 1   From            32016 non-null  object
 2   To              32016 non-null  object
 3   Subject         32016 non-null  object
 4   Body            32016 non-null  object
 5   processed_text  32016 non-null  object
dtypes: object(6)
memory usage: 1.7+ MB


In [71]:
df.dtypes

Date              object
From              object
To                object
Subject           object
Body              object
processed_text    object
dtype: object

In [72]:
# Ensure TensorFlow is installed
%pip install tensorflow==2.10.0

# Ensure TensorFlow is imported correctly
try:
	from tensorflow.keras.preprocessing.text import Tokenizer
	from tensorflow.keras.preprocessing.sequence import pad_sequences

	# Tokenization for LSTM
	tokenizer = Tokenizer(num_words=5000)
	tokenizer.fit_on_texts(df['processed_text'])
	sequences = tokenizer.texts_to_sequences(df['processed_text'])
	X_lstm = pad_sequences(sequences, maxlen=50)
except ImportError as e:
	print("Error importing TensorFlow:", e)
	print("Please ensure TensorFlow is installed correctly and is compatible with your system.")

Note: you may need to restart the kernel to use updated packages.
Error importing TensorFlow: No module named 'tensorflow'
Please ensure TensorFlow is installed correctly and is compatible with your system.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.10.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.10.0


In [73]:
# Load dataset with proper encoding
df = pd.read_excel(r'E:\OM Research Project\Combined_Data.xlsx')

# Print dataset size
print("Dataset Shape:", df.shape)  # Should print (578, X) if all rows are loaded
print(df.head())  # Show first few rows

Dataset Shape: (55436, 5)
                                    Date  \
0  Tue, 19 Nov 2024 08:13:09 +0000 (UTC)   
1  Wed, 20 Nov 2024 06:13:07 +0000 (UTC)   
2  Tue, 19 Nov 2024 06:13:07 +0000 (UTC)   
3  Mon, 18 Nov 2024 14:33:15 +0000 (UTC)   
4  Fri, 29 Nov 2024 01:50:12 +0000 (UTC)   

                                                From  \
0  Career Navigator 2024* <newsletters-noreply@li...   
1  Work From Home Jobs <newsletters-noreply@linke...   
2  Work From Home Jobs <newsletters-noreply@linke...   
3            LinkedIn <updates-noreply@linkedin.com>   
4      Aarushi from foundit <info@alerts.foundit.in>   

                                   To  \
0  Ajay Mane <ajaymane5885@gmail.com>   
1  Ajay Mane <ajaymane5885@gmail.com>   
2  Ajay Mane <ajaymane5885@gmail.com>   
3  Ajay Mane <ajaymane5885@gmail.com>   
4              ajaymane5885@gmail.com   

                                             Subject  \
0  Infosys Walk-In for IT Operations | 21st Nov 2024   
1  Genpact Jo

In [85]:
# Check if the dataset is loading properly
print("Total Rows in Dataset:", df.shape[0])

# Define spam keywords
spam_keywords = ["unsubscribe", "win", "click here", "free", "claim", "urgent", "limited offer", 
                     "winner", "guaranteed", "risk-free", "exclusive", "money", "claim your prize", 
                     "offer expires", "free trial", "credit card", "cash prize", "unsecured", "no payment required"]


# Function to classify emails
def classify_email(text):
    if pd.isna(text):  # Handle missing values
        return "ham"
    text = text.lower()  # Convert to lowercase
    if any(word in text for word in spam_keywords):
        return "spam"
    return "ham"

# Apply classification to ALL rows (not just one row)
df["category"] = df.apply(lambda row: classify_email(str(row["Subject"]) + " " + str(row["Body"])), axis=1)

# Convert category to numerical (ham=0, spam=1)
df["category"] = df["category"].map({"ham": 0, "spam": 1})

# Verify all rows are classified
print(df[["Subject", "Body", "category"]].head())  # Show first few rows
print("Total Rows Classified:", df.shape[0])  # Should be 578+ rows

# Save the classified dataset
df.to_csv("classified_emails.csv", index=False)

Total Rows in Dataset: 55436
                                             Subject  \
0  Infosys Walk-In for IT Operations | 21st Nov 2024   
1  Genpact Jobs Gurgaon for Freshers| Best Opport...   
2  Genpact Jobs Noida for Freshers| Best Opportun...   
3  Vaibhav Kadam just posted something that might...   
4             Hunt for jobs the right way, Jobseeker   

                                                Body  category  
0  Career Navigator 2024*By AMISHA BHATTBy AMISHA...         1  
1  Work From Home JobsBy Work From Home/ Experien...         1  
2  Work From Home JobsBy Work From Home/ Experien...         1  
3  ----------------------------------------      ...         1  
4  <!DOCTYPE html><html xmlns:v="urn:schemas-micr...         1  
Total Rows Classified: 55436


In [86]:
df.category.value_counts()

category
1    32790
0    22646
Name: count, dtype: int64

In [87]:
df.head()

,Date,From,To,Subject,Body,category
0,"Tue, 19 Nov 2024 08:13:09 +0000 (UTC)",Career Navigator 2024* <newsletters-noreply@li...,Ajay Mane <ajaymane5885@gmail.com>,Infosys Walk-In for IT Operations | 21st Nov 2024,Career Navigator 2024*By AMISHA BHATTBy AMISHA...,1
1,"Wed, 20 Nov 2024 06:13:07 +0000 (UTC)",Work From Home Jobs <newsletters-noreply@linke...,Ajay Mane <ajaymane5885@gmail.com>,Genpact Jobs Gurgaon for Freshers| Best Opport...,Work From Home JobsBy Work From Home/ Experien...,1
2,"Tue, 19 Nov 2024 06:13:07 +0000 (UTC)",Work From Home Jobs <newsletters-noreply@linke...,Ajay Mane <ajaymane5885@gmail.com>,Genpact Jobs Noida for Freshers| Best Opportun...,Work From Home JobsBy Work From Home/ Experien...,1
3,"Mon, 18 Nov 2024 14:33:15 +0000 (UTC)",LinkedIn <updates-noreply@linkedin.com>,Ajay Mane <ajaymane5885@gmail.com>,Vaibhav Kadam just posted something that might...,---------------------------------------- ...,1
4,"Fri, 29 Nov 2024 01:50:12 +0000 (UTC)",Aarushi from foundit <info@alerts.foundit.in>,ajaymane5885@gmail.com,"Hunt for jobs the right way, Jobseeker","<!DOCTYPE html><html xmlns:v=""urn:schemas-micr...",1


In [88]:
# Function to automatically label emails based on keywords
def auto_label(subject, body):
    # Convert to lowercase for uniformity
    subject = str(subject).lower()
    body = str(body).lower()
    

    # Spam keywords
    spam_keywords = ["unsubscribe", "win", "click here", "free", "claim", "urgent", "limited offer", 
                     "winner", "guaranteed", "risk-free", "exclusive", "money", "claim your prize", 
                     "offer expires", "free trial", "credit card", "cash prize", "unsecured", "no payment required"]
    if any(keyword in subject or keyword in body for keyword in spam_keywords):
        return "Spam"

    # Promotion keywords
    promo_keywords = ["offer", "deal", "discount", "sale", "coupon", "promo", "clearance", "bargain", 
                      "flash sale", "hot deal", "special offer", "limited time", "today only", "big discount", 
                      "buy now", "shop today", "online exclusive", "exclusive deal", "seasonal sale"]
    if any(keyword in subject or keyword in body for keyword in promo_keywords):
        return "Promotion"

    # Social keywords
    social_keywords = ["friend request", "like", "comment", "follow", "connection", "new follower", 
                       "your post", "social invite", "message request", "share your status", "new like", 
                       "new comment", "friend suggestion", "tagged in photo", "social media update", 
                       "profile update", "join the group"]
    if any(keyword in subject or keyword in body for keyword in social_keywords):
        return "Social"

    # Update keywords
    update_keywords = ["update", "newsletter", "report", "summary", "digest", "alert", "news", 
                       "update notification", "policy update", "account update", "system update", "upgrade", 
                       "change", "announcement", "new feature", "release notes", "latest news"]
    if any(keyword in subject or keyword in body for keyword in update_keywords):
        return "Updates"

    # Default to Primary (Ham)
    return "Primary"

# Apply the automatic labeling function
df['Label'] = df.apply(lambda row: auto_label(row['Subject'], row['Body']), axis=1)

# Display some labeled data samples to verify
df[['From', 'Subject', 'Body', 'Label']].head(20)
# Display the updated dataset with the new labels
# Create a new dataframe with the new column 'Label'
df1 = df.copy()

# Display the updated dataset with the new labels
df1.head()

,Date,From,To,Subject,Body,category,Label
0,"Tue, 19 Nov 2024 08:13:09 +0000 (UTC)",Career Navigator 2024* <newsletters-noreply@li...,Ajay Mane <ajaymane5885@gmail.com>,Infosys Walk-In for IT Operations | 21st Nov 2024,Career Navigator 2024*By AMISHA BHATTBy AMISHA...,1,Spam
1,"Wed, 20 Nov 2024 06:13:07 +0000 (UTC)",Work From Home Jobs <newsletters-noreply@linke...,Ajay Mane <ajaymane5885@gmail.com>,Genpact Jobs Gurgaon for Freshers| Best Opport...,Work From Home JobsBy Work From Home/ Experien...,1,Spam
2,"Tue, 19 Nov 2024 06:13:07 +0000 (UTC)",Work From Home Jobs <newsletters-noreply@linke...,Ajay Mane <ajaymane5885@gmail.com>,Genpact Jobs Noida for Freshers| Best Opportun...,Work From Home JobsBy Work From Home/ Experien...,1,Spam
3,"Mon, 18 Nov 2024 14:33:15 +0000 (UTC)",LinkedIn <updates-noreply@linkedin.com>,Ajay Mane <ajaymane5885@gmail.com>,Vaibhav Kadam just posted something that might...,---------------------------------------- ...,1,Spam
4,"Fri, 29 Nov 2024 01:50:12 +0000 (UTC)",Aarushi from foundit <info@alerts.foundit.in>,ajaymane5885@gmail.com,"Hunt for jobs the right way, Jobseeker","<!DOCTYPE html><html xmlns:v=""urn:schemas-micr...",1,Spam


In [89]:
df['Label'].value_counts()

Label
Spam         32790
Primary      15784
Updates       2507
Promotion     2281
Social        2074
Name: count, dtype: int64

In [90]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [91]:
import pandas as pd
import re

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

df['processed_text'] = df['Body'].apply(preprocess_text)  # Ensure correct column name

In [92]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(df['processed_text'])

In [93]:
X.shape

(55436, 677732)

In [94]:
Y = df['Label'].values

In [95]:
Y

array(['Spam', 'Spam', 'Spam', ..., 'Spam', 'Primary', 'Spam'],
      dtype=object)

In [96]:
import pandas as pd

# Sample email data (replace with your actual data or load from Excel)
data = {
    'Subject': [
        "Congratulations! You won a cash prize", 
        "Flash Sale - 50% OFF today only!", 
        "Your friend liked your post", 
        "Weekly newsletter update", 
        "Meeting scheduled for Monday"
    ],
    'Body': [
        "Click here to claim your prize. No payment required.",
        "Don't miss our hot deal! Limited time offer, shop now.",
        "John commented on your post. View it on your profile.",
        "Here is your weekly newsletter with all the latest news.",
        "Reminder: Team meeting is scheduled for 10 AM Monday."
    ]
}

df = pd.DataFrame(data)

# Define keyword lists
spam_keywords = [
    "unsubscribe", "win", "click here", "free", "claim", "urgent", "limited offer", 
    "winner", "guaranteed", "risk-free", "exclusive", "money", "claim your prize", 
    "offer expires", "free trial", "credit card", "cash prize", "unsecured", 
    "no payment required"
]

promo_keywords = [
    "offer", "deal", "discount", "sale", "coupon", "promo", "clearance", "bargain", 
    "flash sale", "hot deal", "special offer", "limited time", "today only", 
    "big discount", "buy now", "shop today", "online exclusive", "exclusive deal", 
    "seasonal sale"
]

social_keywords = [
    "friend request", "like", "comment", "follow", "connection", "new follower", 
    "your post", "social invite", "message request", "share your status", "new like", 
    "new comment", "friend suggestion", "tagged in photo", "social media update", 
    "profile update", "join the group"
]

update_keywords = [
    "update", "newsletter", "report", "summary", "digest", "alert", "news", 
    "update notification", "policy update", "account update", "system update", 
    "upgrade", "change", "announcement", "new feature", "release notes", "latest news"
]

primary_keywords = [
    "meeting", "appointment", "invoice", "schedule", "reminder", "password", 
    "receipt", "confirmation", "team", "project", "task", "follow-up"
]

# Function to classify emails
def classify_email(subject, body):
    if not isinstance(subject, str):
        subject = ''
    if not isinstance(body, str):
        body = ''
    
    content = (subject + ' ' + body).lower()
    
    if any(keyword in content for keyword in spam_keywords):
        return "Spam"
    elif any(keyword in content for keyword in promo_keywords):
        return "Promotion"
    elif any(keyword in content for keyword in social_keywords):
        return "Social"
    elif any(keyword in content for keyword in update_keywords):
        return "Updates"
    elif any(keyword in content for keyword in primary_keywords):
        return "Primary"
    else:
        return "Primary"  # default fallback

# Apply classification
df['Label'] = df.apply(lambda row: classify_email(row['Subject'], row['Body']), axis=1)

# Display results
print(df[['Subject', 'Body', 'Label']])


                                 Subject  \
0  Congratulations! You won a cash prize   
1       Flash Sale - 50% OFF today only!   
2            Your friend liked your post   
3               Weekly newsletter update   
4           Meeting scheduled for Monday   

                                                Body      Label  
0  Click here to claim your prize. No payment req...       Spam  
1  Don't miss our hot deal! Limited time offer, s...  Promotion  
2  John commented on your post. View it on your p...     Social  
3  Here is your weekly newsletter with all the la...    Updates  
4  Reminder: Team meeting is scheduled for 10 AM ...    Primary  


In [97]:
# Load dataset with proper encoding
df = pd.read_excel(r'E:\OM Research Project\Combined_Data.xlsx', engine='openpyxl')
df.dropna(inplace=True)

# Define keyword lists
spam_keywords = [
    "unsubscribe", "win", "click here", "free", "claim", "urgent", "limited offer", 
    "winner", "guaranteed", "risk-free", "exclusive", "money", "claim your prize", 
    "offer expires", "free trial", "credit card", "cash prize", "unsecured", 
    "no payment required"
]

promo_keywords = [
    "offer", "deal", "discount", "sale", "coupon", "promo", "clearance", "bargain", 
    "flash sale", "hot deal", "special offer", "limited time", "today only", 
    "big discount", "buy now", "shop today", "online exclusive", "exclusive deal", 
    "seasonal sale"
]

social_keywords = [
    "friend request", "like", "comment", "follow", "connection", "new follower", 
    "your post", "social invite", "message request", "share your status", "new like", 
    "new comment", "friend suggestion", "tagged in photo", "social media update", 
    "profile update", "join the group"
]

update_keywords = [
    "update", "newsletter", "report", "summary", "digest", "alert", "news", 
    "update notification", "policy update", "account update", "system update", 
    "upgrade", "change", "announcement", "new feature", "release notes", "latest news"
]

primary_keywords = [
    "meeting", "appointment", "invoice", "schedule", "reminder", "password", 
    "receipt", "confirmation", "team", "project", "task", "follow-up"
]

# Function to classify emails
def classify_email(subject, body):
    if not isinstance(subject, str):
        subject = ''
    if not isinstance(body, str):
        body = ''
    
    content = (subject + ' ' + body).lower()
    
    if any(keyword in content for keyword in spam_keywords):
        return "Spam"
    elif any(keyword in content for keyword in promo_keywords):
        return "Promotion"
    elif any(keyword in content for keyword in social_keywords):
        return "Social"
    elif any(keyword in content for keyword in update_keywords):
        return "Updates"
    elif any(keyword in content for keyword in primary_keywords):
        return "Primary"
    else:
        return "Primary"  # default fallback

# Apply classification
df['Label'] = df.apply(lambda row: classify_email(row['Subject'], row['Body']), axis=1)

# Display results
print(df[['Subject', 'Body', 'Label']])


                                                 Subject  \
0      Infosys Walk-In for IT Operations | 21st Nov 2024   
1      Genpact Jobs Gurgaon for Freshers| Best Opport...   
2      Genpact Jobs Noida for Freshers| Best Opportun...   
3      Vaibhav Kadam just posted something that might...   
4                 Hunt for jobs the right way, Jobseeker   
...                                                  ...   
55424  Kishor Khatau Thacker shared "Dignitaries Accl...   
55426  Dear Sanjay Devgonda Patil, aim to diversify y...   
55428  Top 6 posts on Facebook: posts from Raju Patil...   
55429  vir sanghvi posted: If this guy was a Palestin...   
55435  Lets OTT Tweeted: #ItluMaredumilliPrajaneekam ...   

                                                    Body Label  
0      Career Navigator 2024*By AMISHA BHATTBy AMISHA...  Spam  
1      Work From Home JobsBy Work From Home/ Experien...  Spam  
2      Work From Home JobsBy Work From Home/ Experien...  Spam  
3      ------------

In [98]:
df['Label'].value_counts()

Label
Spam         25094
Primary       3457
Promotion     1419
Updates       1388
Social         658
Name: count, dtype: int64